# Load training set and train paragraph vectors
Note: the paragraph vector model has been trained and is downloaded in the `prepare_feature_extraction()` function.

Retraining is therefore not needed, but optional

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# If you need fully deterministic results between runs, set the following environment value prior to launching jupyter.
# See comment in sherlock.features.paragraph_vectors.infer_paragraph_embeddings_features for more info.
%env PYTHONHASHSEED

'13'

In [3]:
import multiprocessing as mp
import sys

from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from pyarrow.parquet import ParquetFile

from sherlock import helpers
from sherlock.features.paragraph_vectors import (
    initialise_nltk,
    tagcol_paragraph_embeddings_features,
    train_paragraph_embeddings_features
)
from sherlock.features.preprocessing import convert_string_lists_to_lists
from sherlock.functional import extract_features_to_csv

In [4]:
print(f'Started at {datetime.now()}')

Started at 2022-02-10 19:51:19.513751


## Download and read in raw data


In [5]:
helpers.download_data()

Data was downloaded.


In [6]:
train_samples = pd.read_parquet('../data/data/raw/train_values.parquet')
train_labels = pd.read_parquet('../data/data/raw/train_labels.parquet')

In [7]:
train_samples_converted, y_train = convert_string_lists_to_lists(train_samples, train_labels, "values", "type")

100%|██████████| 412059/412059 [01:47<00:00, 3841.46it/s] 


## Train Doc2Vec

In [8]:
initialise_nltk()

Initialised NLTK, process took 0:00:00.340159 seconds.


[nltk_data] Downloading package punkt to /Users/madelon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/madelon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
samples = train_samples_converted.dropna()
print(f'Samples: {type(samples)}, length={len(samples)}')

labels = train_labels.values.flatten()
print(f'Labels:  {type(labels)}, length={len(labels)}')

Samples: <class 'pandas.core.series.Series'>, length=412059
Labels:  <class 'numpy.ndarray'>, length=412059


In [10]:
start = datetime.now()

print('Tagging columns')
cols = tagcol_paragraph_embeddings_features(samples, labels)

print(f'Tagged Columns Doc2Vec Model, process took {datetime.now() - start} seconds.')

Tagging columns
Tagged Columns Doc2Vec Model, process took 0:07:04.637667 seconds.


In [14]:
start = datetime.now()

vec_dim = 400
print(f'Training Doc2Vec model in {vec_dim} dimensions')

train_paragraph_embeddings_features(cols, vec_dim)

print(f'Trained Doc2Vec Model, {vec_dim} dim, process took {datetime.now() - start} seconds.')

Training Doc2Vec model in 400 dimensions
Trained Doc2Vec Model, 400 dim, process took 0:22:22.303905 seconds.


In [15]:
print(f'Finished at {datetime.now()}')

Finished at 2022-02-09 19:38:41.865764
